In [1]:
import os
import shutil
import subprocess
import re
import operator
import pandas as pd
import pickle

from xml.dom.minidom import Element
from androguard.core.bytecodes import apk
from androguard.core.bytecodes.dvm import DalvikVMFormat
from androguard.core.analysis.analysis import Analysis
from androguard.decompiler.decompiler import DecompilerJADX
from androguard.core.androconf import show_logging

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Using Theano backend.


In [2]:
global_perms = []
global_intents = []
global_apis = []

In [13]:
class feat_ex():
    def __init__(self,folder_path):
        self.folder_path = folder_path
        self.perms_list = []
        self.intent_list = []
        self.api_list = []
        self.file_list=[]
        self.file_paths()
        
    def file_paths(self):
        apk_folder = os.listdir(self.folder_path)
        print (self.folder_path)
        for ix in range(len(apk_folder)):
            if apk_folder[ix]!='.DS_Store':
                self.file_list.append(os.path.join(self.folder_path,apk_folder[ix]))
        return
    def populate_perms(self):
        i=0
        for ix in self.file_list:
            print i,'---->',ix
            i+=1
            app = apk.APK(ix)
            perms = app.get_permissions()
#             print ix,':\n',perms
            global_perms.extend(sorted(list(set(perms))))
#         self.perms_list = sorted(set(self.perms_list))
                
    def populate_apis(self):
        j=0
        for ix in self.file_list:
            print j,'--->',ix
            j+=1
            try:
                app= apk.APK(ix)
                application_dex = DalvikVMFormat(app.get_dex())
                application_x = Analysis(application_dex)
                methods = set()
                cs = [cc.get_name() for cc in application_dex.get_classes()]

                for method in application_dex.get_methods():
                    g = application_x.get_method(method)

                    if method.get_code() == None:
                        continue

                    for i in g.get_basic_blocks().get():
                        for ins in i.get_instructions():
                            output = ins.get_output()
                            match = re.search(r'(L[^;]*;)->[^\(]*\([^\)]*\).*', output)
                            if match and match.group(1) not in cs:
                                methods.add(match.group())

                apis = list(methods)
                apk_api_list = []
                for api in apis:
                    start_index = api.find('>')
                    end_index = api.find('(')
                    start_index+=1
                    api = api[start_index:end_index] + '()' 
                    apk_api_list.append(api)
                global_apis.extend(sorted(list(set(apk_api_list))))
            except:
                print 'error'
                continue



In [10]:
class feat_mat():
    def __init__(self,file_list):
        self.file_list=file_list
        
    def perm_matrix(self):
        matrix = []
        i=0
        for ix in self.file_list:
            print i,'---->',ix
            i+=1
            app = apk.APK(ix)
            perms = app.get_permissions()
            row=[0]*len(global_perms)
            for ix in perms:
                if ix in global_perms:
                    row[global_perms.index(ix)]=1
            matrix.append(row)
        return matrix
    def api_matrix(self):
        matrix = []
        j=0
        for ix in self.file_list:
            print j,'----->',ix
            j+=1
            try:
                app= apk.APK(ix)
                application_dex = DalvikVMFormat(app.get_dex())
                application_x = Analysis(application_dex)
                methods = set()
                cs = [cc.get_name() for cc in application_dex.get_classes()]

                for method in application_dex.get_methods():
                    g = application_x.get_method(method)

                    if method.get_code() == None:
                        continue

                    for i in g.get_basic_blocks().get():
                        for ins in i.get_instructions():
                            output = ins.get_output()
                            match = re.search(r'(L[^;]*;)->[^\(]*\([^\)]*\).*', output)
                            if match and match.group(1) not in cs:
                                methods.add(match.group())

                apis = list(methods)
                apk_api_list = []
                for api in apis:
                    start_index = api.find('>')
                    end_index = api.find('(')
                    start_index+=1
                    api = api[start_index:end_index] + '()' 
                    apk_api_list.append(api)
                row=[0]*len(global_apis)
                for ix in apk_api_list:
                    if ix in global_apis:
                        row[global_apis.index(ix)]=1
                matrix.append(row)
            except:
                print 'error'
                continue
        return matrix


In [7]:
a = feat_ex('/Users/apple/Desktop/ml/apks')
a.populate_apis()
print len(global_apis),len(global_apis[0])

/Users/apple/Desktop/ml/apks
0 ---> /Users/apple/Desktop/ml/apks/1d09be5cc612b24fe83c3f1b37c88087d778dd5fd3f33e4f09df8eda990e41be
1 ---> /Users/apple/Desktop/ml/apks/1c24f140d022f677f420a8c73d25825a09a8f0a457570c97f3faaeb141c455e9
2 ---> /Users/apple/Desktop/ml/apks/0b9678ef94e8dcc2bb4d3b2fa5a75bb9ae4f7ef464784d1a2509e003da699707
3 ---> /Users/apple/Desktop/ml/apks/2bba69f5994c871380d5633e37305e7c09db17bb3ebaeeb94163625609d866d9
4 ---> /Users/apple/Desktop/ml/apks/2c86c0f14b3eeedc487b4cf23ca5a2243f173e435eb587a68bd617fe9d891159
5 ---> /Users/apple/Desktop/ml/apks/2ed7a5aa90f7d3163341b5450bfdf36c80545805dc28f37e58b6b10bb3bfb92c
6 ---> /Users/apple/Desktop/ml/apks/1cf3932ecdc72f170d7a72a5108820487c55afe9f9d978b32ed1ffc3991e2c3e
7 ---> /Users/apple/Desktop/ml/apks/1b3b747592733d102c9995325f20c734e213a784f3f90aabc4602d74523b64bd
8 ---> /Users/apple/Desktop/ml/apks/2a93eb58c737572dd375a36f8a5397140c815dcae05dc80dafaad4b3981a91b6
9 ---> /Users/apple/Desktop/ml/apks/1eef977709780add638465e1ad

81 ---> /Users/apple/Desktop/ml/apks/0c221fe6be180e5c694c84cd192b62db995e01e706982444d0ac3c837e4cdbb4
82 ---> /Users/apple/Desktop/ml/apks/00c8de6b31090c32b65f8c30d7227488d2bce5353b31bedf5461419ff463072d
83 ---> /Users/apple/Desktop/ml/apks/1d326ba9c19e14a5b5cc22df85596c3a3c6394c49d1b64ad144f601ee06cde79
84 ---> /Users/apple/Desktop/ml/apks/2a618d9d496c21d944bcd9700a8ac18213a88d2ef220af6f130a04b145307650
85 ---> /Users/apple/Desktop/ml/apks/1f7f7f14ca4eecd51f5c52f9034bc9a2ad859c2ea511d2c14fe20aa7af529a8c
86 ---> /Users/apple/Desktop/ml/apks/0ce6a7f59ce12d472e6949f14c142abe68b82b03c4360a65495faa2931aad52d
87 ---> /Users/apple/Desktop/ml/apks/1b3372d4243776ef09d50761aa53aa2fe486d468ff2fa31b46363a0c96929eaa
88 ---> /Users/apple/Desktop/ml/apks/2ef98dbdd9ac7b92f7771a62e8e8505d9888901f3bdb02919f7837a929d9cff1
89 ---> /Users/apple/Desktop/ml/apks/1ed6ac180386d0fb61ae05e832f475228ca414c34f3f9c6c0cda1c63d92dd9f5
90 ---> /Users/apple/Desktop/ml/apks/5b72e14c1bdca5e2def1915deb3b05d1a64c20c8912e4

161 ---> /Users/apple/Desktop/ml/apks/3b00973af1764f647382ed292c73349c3337b61f0862b2069513f0946a05d0b3
162 ---> /Users/apple/Desktop/ml/apks/0c5d78b0e0d0693c8aa9478e6fef05760e45d6ee809aa75c19125eb9e9f93ba4
163 ---> /Users/apple/Desktop/ml/apks/1fed19d4ca265fc5458a83fa057499760155c5dbfbaed6802663aef263c63963
164 ---> /Users/apple/Desktop/ml/apks/03b135be43ac95338afa14cad603d5340de1e96a6516f38024bc5016605e061a
165 ---> /Users/apple/Desktop/ml/apks/1b6651756f5d460cc30fb618e61ff04582c24ddbafe55288897a3444bd4d827d
166 ---> /Users/apple/Desktop/ml/apks/0ccc7f9cade5aa4a8d462bb5ca01f1ea657739ac04be8aa40c5703a976499729
167 ---> /Users/apple/Desktop/ml/apks/1e0de6bc270ce48afb50b6ef62924281b3767d84b9fcc18c58c68041394f7479
168 ---> /Users/apple/Desktop/ml/apks/0df7fb1797024888b7ca2b2c70e0b562037962d46d1c1d2ce50e1bcf62bb5886
169 ---> /Users/apple/Desktop/ml/apks/1f8b333004cd82580a5481900e9e110b995557c2200758c7d9483bafe7032f94
170 ---> /Users/apple/Desktop/ml/apks/2f0239ddd3ad18dab194e684e991d6bac5b

241 ---> /Users/apple/Desktop/ml/apks/2aaf6378b79c13fa3c5880e9ecebb594430e786f405427ba0ae1de15d3907eb2
242 ---> /Users/apple/Desktop/ml/apks/2f119ef66cc3fccab2beb77c4f5273be33b8ffe36b598e58a3846277cc80bc96
243 ---> /Users/apple/Desktop/ml/apks/3abbb51e6386f9d4b2d8ce97444561d7080946d82f1cba254f98b0edfd46a163
244 ---> /Users/apple/Desktop/ml/apks/0e399390097a1dbe6441fabccbdf7ac70f9cf5a5692ead509d4255d44e79a626
245 ---> /Users/apple/Desktop/ml/apks/2aebd2a3730cb562a316fbb8c31d22b47ea5e993a6b3d8d5bea77bb3859c3327
246 ---> /Users/apple/Desktop/ml/apks/0c8890bd550dd21cc5b0feb9ad101984e194719b2656ac904b7aefb35765b15f
247 ---> /Users/apple/Desktop/ml/apks/1de6dd8784bef19d3c52a52db48ebdcf3b02a9a936546af7b77627918fe47030
248 ---> /Users/apple/Desktop/ml/apks/1f0133014fac23a55fec3f56c9a09a2f0b7d02fc8f0c6c5b703397562e57098f
249 ---> /Users/apple/Desktop/ml/apks/3a2bd4860eba4a72257588566a8c5825261a6c3a20e704cdb2bcc28ab22a7f3e
250 ---> /Users/apple/Desktop/ml/apks/1dc1219c45ca4027d41bab6211b10148faa

321 ---> /Users/apple/Desktop/ml/apks/2f360c0458e300a92bd44e9c2b7aff25fe295121a3c2f99b718c8e37de1963a2
322 ---> /Users/apple/Desktop/ml/apks/0be2a032fcfc07fca6c7c9f3fda1424743999b050482e66695fa4bab4b1fa0b9
323 ---> /Users/apple/Desktop/ml/apks/1aa624fff0826a46b8877bc9f55122f1cdb416329daaf774d05479b0ceed26e5
324 ---> /Users/apple/Desktop/ml/apks/05b87d92c45111cb5f22562cfdad72d10ea1955edfce65ec64f0bec8853d2e19
325 ---> /Users/apple/Desktop/ml/apks/0a282ade29d49e2c01cadd5b6a65b6434f5ad7c96d3360e8c8831cfbc6d8cedf
326 ---> /Users/apple/Desktop/ml/apks/2e24b9e064e812a04ce85678cd7e736492c5a9ec0a73d5c8900b01e692ec0bee
327 ---> /Users/apple/Desktop/ml/apks/0dcad1b75b528e1f20b40e4ef70487534db04c4592a7f9e93183f352ed49fa21
328 ---> /Users/apple/Desktop/ml/apks/0ffc91c7f144c802033128530cb87a3acf201020a4901dda1f2db9c4418ea793
329 ---> /Users/apple/Desktop/ml/apks/1a76595f4e09d4bc03a5f77bb3b235c6a0974975e21da85e0c767cf84ae096a9
330 ---> /Users/apple/Desktop/ml/apks/2c0c697f1adb39d2121ca123bdcbe82824b

402 ---> /Users/apple/Desktop/ml/apks/1ed12f16e504218209e5ceb5f51bde88c585709b8babf87bb89742e0d1203693
403 ---> /Users/apple/Desktop/ml/apks/02e112947aa19a577fd9d825531bd74797bbf5825a74e9918d4027bbd24bb49b
404 ---> /Users/apple/Desktop/ml/apks/2ff6cf239fbae1e7768bf8acb6a0f7491e1cbd9a48bb6ae5de19050de49be0f1
405 ---> /Users/apple/Desktop/ml/apks/02cc86e73be61180fd66ac9625f655217757291ba255adbe1efeca2d1f2803ec
406 ---> /Users/apple/Desktop/ml/apks/1d22924bbe5dce7696e18d880482b63ce19ca0746f8671aaec865cce143f6e6f
407 ---> /Users/apple/Desktop/ml/apks/2bff76df09ffa038c77ad44ecced13f1125c258b2fa60fd229002f7f1dafda15
408 ---> /Users/apple/Desktop/ml/apks/0eff4ed561b34415051fb462dc998725ddbb6abd985cbabdac2c714b9f1b0da4
409 ---> /Users/apple/Desktop/ml/apks/1a1bbe402c430afed6c97c125453091da9da26bbd5418bb36659b3614c2c898a
410 ---> /Users/apple/Desktop/ml/apks/3bdefb33f0b86a2ccf2b5a0f2c9b06718a92ca8b5ad8bf6fc57a2e47328b3b57
411 ---> /Users/apple/Desktop/ml/apks/3b849210c61e270be93fde230c8f2ba67bb

482 ---> /Users/apple/Desktop/ml/apks/2c2458fdc1112a6175d6ff4b4388a8f01ef0f02d9f95e101eaabb0bd2903b377
483 ---> /Users/apple/Desktop/ml/apks/1eeb97dd6c57f45f54d5318e62dc63963a062f5c9d78279e353c33bf6748412e
484 ---> /Users/apple/Desktop/ml/apks/1e4f10fb9d2622c1a956d28a8bdf9ca89aabd90da48de5d47103b9f9403513e7
485 ---> /Users/apple/Desktop/ml/apks/2e8f821f70be10803b053e229150fc38fbeca68b4789ca059a1197fd63ef7733
486 ---> /Users/apple/Desktop/ml/apks/2cec2b77e0d10e39956730f38a89761293aae7da4a3299d9d0d69a83f7f68ef4
487 ---> /Users/apple/Desktop/ml/apks/2bdc0e959fd300abe76a974b01535d654b08573300c1b9e4863edd1b555d19dc
488 ---> /Users/apple/Desktop/ml/apks/1a098e5cec999248f284ad035ede5c2035e65f981e3f24c2c415abc3fbfe7833
489 ---> /Users/apple/Desktop/ml/apks/1b825312a360af320c9ce3992140e84fd9fc25b68b6c8c3987cc0027020038db
490 ---> /Users/apple/Desktop/ml/apks/0ca64e19e09d8af20a7a9269aca259da3406c5fd1c3627f92fb601af2c2d7d9a
491 ---> /Users/apple/Desktop/ml/apks/01be90f0119b722d0012c269a89dd548138

In [5]:
b = feat_ex('/Users/apple/Desktop/ml/crawler/benign/')
b.populate_apis()
print len(global_apis),len(global_apis[0])

/Users/apple/Desktop/ml/crawler/benign/
0 ---> /Users/apple/Desktop/ml/crawler/benign/com.myfitnesspal.android_v6.37.2-10948_Android-5.0.apk
1 ---> /Users/apple/Desktop/ml/crawler/benign/com.rhmsoft.fm_v2.7.7-20770706_Android-4.0.apk
2 ---> /Users/apple/Desktop/ml/crawler/benign/com.jio.join_v5.0.6.multireg-5000604_Android-4.1.apk
3 ---> /Users/apple/Desktop/ml/crawler/benign/org.freeandroidtools.root_checker_v3.1.1-2293_Android-4.0.apk
4 ---> /Users/apple/Desktop/ml/crawler/benign/com.lookout_v10.22-5091e54-10220069_Android-4.1.apk
5 ---> /Users/apple/Desktop/ml/crawler/benign/com.urbandroid.lux_v8.3-275_Android-4.0.apk
6 ---> /Users/apple/Desktop/ml/crawler/benign/com.naturalmotion.csrracing_v5.0.1-21363_Android-4.0.3.apk
7 ---> /Users/apple/Desktop/ml/crawler/benign/com.amazon.mp3_v7.8.4_307080410-307080410_Android-4.4.apk
8 ---> /Users/apple/Desktop/ml/crawler/benign/com.igg.castleclash_de_v1.3.6-1300600_Android-4.1.apk
9 ---> /Users/apple/Desktop/ml/crawler/benign/com.shazam.andro

No handlers could be found for logger "androguard.axml"


error
14 ---> /Users/apple/Desktop/ml/crawler/benign/com.microsoft.office.powerpoint_v16.0.9330.2080-2001479441_Android-4.4.apk
15 ---> /Users/apple/Desktop/ml/crawler/benign/com.google.android.projection.gearhead_v3.2.581533-release-32581533_Android-5.0.apk
16 ---> /Users/apple/Desktop/ml/crawler/benign/com.ea.gp.fifaworld_v3.2.113645-26_Android-4.4.apk
17 ---> /Users/apple/Desktop/ml/crawler/benign/jp.pokemon.koiking_v1.3.5-28_Android-4.1.apk
18 ---> /Users/apple/Desktop/ml/crawler/benign/com.pecana.iptvextreme_v76.0-76_Android-4.1.apk
19 ---> /Users/apple/Desktop/ml/crawler/benign/com.bandainamcoent.narutoblazingna_v2.6.0-75_Android-4.2.apk
20 ---> /Users/apple/Desktop/ml/crawler/benign/com.linecorp.LGTMTMG_v1.45.0-59_Android-4.0.3.apk
21 ---> /Users/apple/Desktop/ml/crawler/benign/com.telltalegames.minecraft100_v1.37-40137_Android-2.3.3.apk
22 ---> /Users/apple/Desktop/ml/crawler/benign/com.gameloft.android.ANMP.GloftSXHM_v2.9.2c-29225_Android-2.3.2.apk
23 ---> /Users/apple/Desktop

92 ---> /Users/apple/Desktop/ml/crawler/benign/com.google.android.apps.subscriptions.red_v1.0.197473691-11373_Android-4.2.apk
93 ---> /Users/apple/Desktop/ml/crawler/benign/com.opera.mini.next.android_v7.5.1-23_Android-1.5.apk
94 ---> /Users/apple/Desktop/ml/crawler/benign/com.Level5.YWWWUS_v2.2.5-4005_Android-4.0.apk
95 ---> /Users/apple/Desktop/ml/crawler/benign/air.com.and.games505.gemsofwar_v3.4.016-3400016_Android-4.1.apk
96 ---> /Users/apple/Desktop/ml/crawler/benign/com.youthhr.phonto_v1.7.15-58_Android-4.0.3.apk
97 ---> /Users/apple/Desktop/ml/crawler/benign/com.waybefore.fastlikeafox_v1.3.7-10360_Android-4.2.apk
98 ---> /Users/apple/Desktop/ml/crawler/benign/com.justunfollow.android_v4.8.1-336_Android-4.0.apk
99 ---> /Users/apple/Desktop/ml/crawler/benign/com.kmplayer_v3.0.17-1228_Android-4.0.3.apk
100 ---> /Users/apple/Desktop/ml/crawler/benign/com.liquidum.rocketvpn_v1.20-305_Android-4.1.apk
101 ---> /Users/apple/Desktop/ml/crawler/benign/org.jw.jwlibrary.mobile_v10.5-24290_

171 ---> /Users/apple/Desktop/ml/crawler/benign/com.gameloft.android.ANMP.GloftDOHM_v3.7.0i-37028_Android-4.0.3.apk
172 ---> /Users/apple/Desktop/ml/crawler/benign/com.epicactiononline.ffxv.ane_v3.29.17.79-17319_Android-4.1.apk
173 ---> /Users/apple/Desktop/ml/crawler/benign/com.fingersoft.hillclimb_v1.37.0-186_Android-4.2.apk
174 ---> /Users/apple/Desktop/ml/crawler/benign/com.sgiggle.production_v4.9.227794-1525958094_Android-4.4.apk
175 ---> /Users/apple/Desktop/ml/crawler/benign/com.google.android.apps.photos_v3.21.0.198453719-2522540_Android-4.4.apk
176 ---> /Users/apple/Desktop/ml/crawler/benign/com.anghami_v3.5.30-888_Android-4.1.apk
177 ---> /Users/apple/Desktop/ml/crawler/benign/pv.player.free_v20.1-402_Android-4.1.apk
error
178 ---> /Users/apple/Desktop/ml/crawler/benign/com.fromthebenchgames.nbamanager15_v4.30.030-430030_Android-4.1.apk
179 ---> /Users/apple/Desktop/ml/crawler/benign/com.yahoo.mobile.client.android.finance_v4.7.4-1330755303_Android-4.4.apk
180 ---> /Users/app

249 ---> /Users/apple/Desktop/ml/crawler/benign/com.cmcm.live_v3.8.76-38761079_Android-4.4.apk
250 ---> /Users/apple/Desktop/ml/crawler/benign/com.azure.authenticator_v6.2.29-119_Android-4.2.apk
251 ---> /Users/apple/Desktop/ml/crawler/benign/com.ubisoft.raymanadventures_v3.4.2-1644_Android-4.1.apk
252 ---> /Users/apple/Desktop/ml/crawler/benign/com.cma.launcher.lite_v3.10.7.1-125_Android-4.0.apk
error
253 ---> /Users/apple/Desktop/ml/crawler/benign/com.imangi.templerun2_v1.48.0-183_Android-4.0.apk
254 ---> /Users/apple/Desktop/ml/crawler/benign/com.javiersantos.mlmanager_v3.3.1-80_Android-4.1.apk
255 ---> /Users/apple/Desktop/ml/crawler/benign/com.magix.android.mmjam_v4.1.11.1-2145235196_Android-4.1.apk
256 ---> /Users/apple/Desktop/ml/crawler/benign/mobi.infolife.uninstaller_v3.3.1-128_Android-4.0.apk
257 ---> /Users/apple/Desktop/ml/crawler/benign/com.ubisoft.hungrysharkworld_v2.8.0-127_Android-4.2.apk
258 ---> /Users/apple/Desktop/ml/crawler/benign/stericson.busybox_v60-210_Android

329 ---> /Users/apple/Desktop/ml/crawler/benign/com.app.downloadmanager_v2.4.54-123_Android-4.1.apk
330 ---> /Users/apple/Desktop/ml/crawler/benign/es.expectro.revelan_v1.4.0-58_Android-2.2.apk
331 ---> /Users/apple/Desktop/ml/crawler/benign/com.king.blossomblast_v55.0.2-157_Android-4.0.3.apk
332 ---> /Users/apple/Desktop/ml/crawler/benign/com.sec.android.easyMover.Agent_v1.6.04.0-160400000_Android-7.0.apk
333 ---> /Users/apple/Desktop/ml/crawler/benign/com.contextlogic.wish_v4.18.5-691_Android-4.1.apk
error
334 ---> /Users/apple/Desktop/ml/crawler/benign/com.estrongs.android.pop.cupcake_v1.6.2.8-106_Android-2.0.apk
335 ---> /Users/apple/Desktop/ml/crawler/benign/com.pixonic.wwr_v3.9.0-10307_Android-4.1.apk
336 ---> /Users/apple/Desktop/ml/crawler/benign/com.yahoo.mobile.client.android.im_v2.10.0-20156_Android-4.4.apk
337 ---> /Users/apple/Desktop/ml/crawler/benign/com.rovio.baba_v2.20.2-2200016_Android-4.1.apk
338 ---> /Users/apple/Desktop/ml/crawler/benign/org.cyanogenmod.oneclick_v1

407 ---> /Users/apple/Desktop/ml/crawler/benign/com.valvesoftware.android.steam.community_v2.3.2-4537696_Android-4.4.apk
408 ---> /Users/apple/Desktop/ml/crawler/benign/com.facebook.mlite_v33.0.0.7.185-110895756_Android-2.3.apk
409 ---> /Users/apple/Desktop/ml/crawler/benign/com.tencent.mobileqq_v7.5.8-818_Android-4.0.3.apk
410 ---> /Users/apple/Desktop/ml/crawler/benign/com.square_enix.android_googleplay.mobiusff_j_v2.0.010-270_Android-2.3.2.apk
411 ---> /Users/apple/Desktop/ml/crawler/benign/com.xvideostudio.videoeditor_v5.0.0-cn-2448_Android-4.0.apk
412 ---> /Users/apple/Desktop/ml/crawler/benign/com.flipkart.android_v6.3-900300_Android-4.1.apk
413 ---> /Users/apple/Desktop/ml/crawler/benign/com.superevilmegacorp.game_v3.3.2-77303-77303_Android-4.1.apk
414 ---> /Users/apple/Desktop/ml/crawler/benign/com.google.android.apps.adm_v2.2.009-20000096_Android-4.0.apk
415 ---> /Users/apple/Desktop/ml/crawler/benign/com.webzen.muorigin.global.google_v2.4.2-242_Android-4.0.3.apk
416 ---> /Use

484 ---> /Users/apple/Desktop/ml/crawler/benign/com.flyersoft.moonreader_v4.5.1-451003_Android-4.0.apk
485 ---> /Users/apple/Desktop/ml/crawler/benign/com.zeptolab.myomnom.free.google_v1.5.3-415010234_Android-4.0.apk
486 ---> /Users/apple/Desktop/ml/crawler/benign/com.duapps.recorder_v1.6.8.1-80_Android-5.0.apk
487 ---> /Users/apple/Desktop/ml/crawler/benign/com.ao.generaladaptiveapps_v1.0.8-9_Android-4.0.apk
488 ---> /Users/apple/Desktop/ml/crawler/benign/com.gameloft.android.ANMP.GloftNOHM_v5.3.1b-53132_Android-4.0.3.apk
489 ---> /Users/apple/Desktop/ml/crawler/benign/com.ki.generaladaptiveapps_v1.0.2-3_Android-4.0.apk
490 ---> /Users/apple/Desktop/ml/crawler/benign/com.miniclip.eightballpool_v3.13.6-1507_Android-4.0.3.apk
491 ---> /Users/apple/Desktop/ml/crawler/benign/com.supercell.boombeach_v34.202-34202_Android-4.0.3.apk
492 ---> /Users/apple/Desktop/ml/crawler/benign/com.jumpgames.rswrb_v34.34.953-34349531_Android-4.0.3.apk
493 ---> /Users/apple/Desktop/ml/crawler/benign/tv.peri

In [28]:
mal_api_mat = np.load('/Users/apple/Desktop/ml/crawler/mal_api_mat.npy')
print mal_api_mat.shape

(505, 61459)


In [4]:
global_apis = list(set(global_api_np.tolist()))

In [26]:
global_apis = list(set(global_apis))
print len(global_apis)

61459


In [14]:
a = feat_ex('/Users/apple/Desktop/ml/apks')
b = feat_ex('/Users/apple/Desktop/ml/crawler/benign/')

/Users/apple/Desktop/ml/apks
/Users/apple/Desktop/ml/crawler/benign/


In [15]:
mal_api_mat = feat_mat(a.file_list).api_matrix()
print len(mal_api_mat),len(mal_api_mat[0])

0 -----> /Users/apple/Desktop/ml/apks/1d09be5cc612b24fe83c3f1b37c88087d778dd5fd3f33e4f09df8eda990e41be
1 -----> /Users/apple/Desktop/ml/apks/1c24f140d022f677f420a8c73d25825a09a8f0a457570c97f3faaeb141c455e9
2 -----> /Users/apple/Desktop/ml/apks/0b9678ef94e8dcc2bb4d3b2fa5a75bb9ae4f7ef464784d1a2509e003da699707
3 -----> /Users/apple/Desktop/ml/apks/2bba69f5994c871380d5633e37305e7c09db17bb3ebaeeb94163625609d866d9
4 -----> /Users/apple/Desktop/ml/apks/2c86c0f14b3eeedc487b4cf23ca5a2243f173e435eb587a68bd617fe9d891159
5 -----> /Users/apple/Desktop/ml/apks/2ed7a5aa90f7d3163341b5450bfdf36c80545805dc28f37e58b6b10bb3bfb92c
6 -----> /Users/apple/Desktop/ml/apks/1cf3932ecdc72f170d7a72a5108820487c55afe9f9d978b32ed1ffc3991e2c3e
7 -----> /Users/apple/Desktop/ml/apks/1b3b747592733d102c9995325f20c734e213a784f3f90aabc4602d74523b64bd
8 -----> /Users/apple/Desktop/ml/apks/2a93eb58c737572dd375a36f8a5397140c815dcae05dc80dafaad4b3981a91b6
9 -----> /Users/apple/Desktop/ml/apks/1eef977709780add638465e1ada5891e76d

79 -----> /Users/apple/Desktop/ml/apks/0c602ee784dd9c06cb8ba7b6a8c983731e53dc8d5e06090d8f0b211444d6cae2
80 -----> /Users/apple/Desktop/ml/apks/2f497c960fdec87068fff7abae7403c4ded94f8bc2125cfb8677c7d52595976f
81 -----> /Users/apple/Desktop/ml/apks/0c221fe6be180e5c694c84cd192b62db995e01e706982444d0ac3c837e4cdbb4
82 -----> /Users/apple/Desktop/ml/apks/00c8de6b31090c32b65f8c30d7227488d2bce5353b31bedf5461419ff463072d
83 -----> /Users/apple/Desktop/ml/apks/1d326ba9c19e14a5b5cc22df85596c3a3c6394c49d1b64ad144f601ee06cde79
84 -----> /Users/apple/Desktop/ml/apks/2a618d9d496c21d944bcd9700a8ac18213a88d2ef220af6f130a04b145307650
85 -----> /Users/apple/Desktop/ml/apks/1f7f7f14ca4eecd51f5c52f9034bc9a2ad859c2ea511d2c14fe20aa7af529a8c
86 -----> /Users/apple/Desktop/ml/apks/0ce6a7f59ce12d472e6949f14c142abe68b82b03c4360a65495faa2931aad52d
87 -----> /Users/apple/Desktop/ml/apks/1b3372d4243776ef09d50761aa53aa2fe486d468ff2fa31b46363a0c96929eaa
88 -----> /Users/apple/Desktop/ml/apks/2ef98dbdd9ac7b92f7771a62e

No handlers could be found for logger "androguard.axml"


90 -----> /Users/apple/Desktop/ml/apks/5b72e14c1bdca5e2def1915deb3b05d1a64c20c8912e4f6924dffed4fd3ad666
91 -----> /Users/apple/Desktop/ml/apks/0ba56840adf155b07bf6ef96b141acd2e5d49f0f17d4bcc9b887b62a39fb06eb
92 -----> /Users/apple/Desktop/ml/apks/0ef389e1da85d91010b64028197b294a9005fbc28c8ec3e742f299c0c6bea5fb
93 -----> /Users/apple/Desktop/ml/apks/1fc57fc187d63eb98df33755cb20ebb9da292c6f667ec265409940ddfc9dffab
94 -----> /Users/apple/Desktop/ml/apks/1eb4eca38f3594c45a0275fc4c775d44ee267cf87b743b5c252ed5a7b5c7a06c
95 -----> /Users/apple/Desktop/ml/apks/00fb209d9f9f1b68b4e13076b555ca1a6da4482f5d1c9af189704e5b5195a09e
96 -----> /Users/apple/Desktop/ml/apks/3ac65392a5e371d1ff5c7a2cbf580a4f3c2a5b36e11c01601d6b38d715c2a74b
97 -----> /Users/apple/Desktop/ml/apks/3a441c39b3b4fee70e34589f6d0073424181620d47ab2c802215e751b49114a3
98 -----> /Users/apple/Desktop/ml/apks/0b89540441c58c7e3e3bc38b457349053494496022119c5153ee1d295ff9bcb8
99 -----> /Users/apple/Desktop/ml/apks/0e357f5fa14ca2fb31d3de9ff

169 -----> /Users/apple/Desktop/ml/apks/1f8b333004cd82580a5481900e9e110b995557c2200758c7d9483bafe7032f94
170 -----> /Users/apple/Desktop/ml/apks/2f0239ddd3ad18dab194e684e991d6bac5b3f9b2ea788584066c3697f5b22fea
171 -----> /Users/apple/Desktop/ml/apks/1e1fa6d2ee545fd5e77729151344e16d8b1a835a1a8d209cb371e534f1caac40
172 -----> /Users/apple/Desktop/ml/apks/1f04474ee6ef6fe893865c66e9f6f12fe7cb09e50ff5e008f4a9c58ac7a51cb6
173 -----> /Users/apple/Desktop/ml/apks/2bb5e30c62e2049ade758cf0375367b02732ce24b4c56bb234b4c4dba3760089
174 -----> /Users/apple/Desktop/ml/apks/2cd54001ba9d0b16da1f6e7f17332ec9cc7dd66a99db4b16746f856b88b75526
175 -----> /Users/apple/Desktop/ml/apks/0ae44d1320be319e2d15a5c83c240bb1fafb28d3f8612b34e0d6adde2315926e
176 -----> /Users/apple/Desktop/ml/apks/0ba73fb2257f2c1794a47c0295750c2b8cb914cbf177bd708e7d10539286d9e1
177 -----> /Users/apple/Desktop/ml/apks/1b66f2c037937fdf4d1677710b506921f875ba8540080ab208978e44e6de1be5
178 -----> /Users/apple/Desktop/ml/apks/2e0d0ae7aa05bac

248 -----> /Users/apple/Desktop/ml/apks/1f0133014fac23a55fec3f56c9a09a2f0b7d02fc8f0c6c5b703397562e57098f
249 -----> /Users/apple/Desktop/ml/apks/3a2bd4860eba4a72257588566a8c5825261a6c3a20e704cdb2bcc28ab22a7f3e
250 -----> /Users/apple/Desktop/ml/apks/1dc1219c45ca4027d41bab6211b10148faa1ab8b3d3015be127960bf723cce56
251 -----> /Users/apple/Desktop/ml/apks/2d2dbc160478c9c4c1a506e806200c5e813dbf2d4f209d25a7541755ea071eb7
252 -----> /Users/apple/Desktop/ml/apks/1eb26d8d7b8bb323a4b97279e0b16707c72434d01102fe5e8f26d6f2e4fc60fa
253 -----> /Users/apple/Desktop/ml/apks/2c842964263a743ae570040a594cbb90781697bc71d0343e9434b1498c109e42
254 -----> /Users/apple/Desktop/ml/apks/0cfa884d0c444a7805d076840038420d8acf957e7ebd491ee334204462f38c2c
255 -----> /Users/apple/Desktop/ml/apks/0dcc348b48e16f5f23ae1a23cbdac96e6c82ac7a6be2dd7f3a5c271904d1967f
256 -----> /Users/apple/Desktop/ml/apks/0cb1212392eef15481eba3bdfcb61be850d372e1329c2da68e73803084045102
257 -----> /Users/apple/Desktop/ml/apks/1e6251fa1bb5b5d

327 -----> /Users/apple/Desktop/ml/apks/0dcad1b75b528e1f20b40e4ef70487534db04c4592a7f9e93183f352ed49fa21
328 -----> /Users/apple/Desktop/ml/apks/0ffc91c7f144c802033128530cb87a3acf201020a4901dda1f2db9c4418ea793
329 -----> /Users/apple/Desktop/ml/apks/1a76595f4e09d4bc03a5f77bb3b235c6a0974975e21da85e0c767cf84ae096a9
330 -----> /Users/apple/Desktop/ml/apks/2c0c697f1adb39d2121ca123bdcbe82824be7f0618ebf551a5e514eedbb2f76b
331 -----> /Users/apple/Desktop/ml/apks/1c29b3010a2e50bca0c5bb45963a74d135addfa206e29785c564fc0a61f1a183
332 -----> /Users/apple/Desktop/ml/apks/0a02157b6acd24011b8d165da2625b7a6c804287701f5e211d296bad2a0a10ce
333 -----> /Users/apple/Desktop/ml/apks/0d5a8ac291f6180102570fccab96fe6ca675ad9466f26d3b3e8bbb619c205c77
334 -----> /Users/apple/Desktop/ml/apks/0e5c06f8f18ead880d28c6b663109ed91469596f1beab347bc870fd639696261
335 -----> /Users/apple/Desktop/ml/apks/0c49a873f66c286996debf6c523c5236b258e0aa48c49c10f211c641b3bcb7c0
336 -----> /Users/apple/Desktop/ml/apks/2d1eee053f84bff

406 -----> /Users/apple/Desktop/ml/apks/1d22924bbe5dce7696e18d880482b63ce19ca0746f8671aaec865cce143f6e6f
407 -----> /Users/apple/Desktop/ml/apks/2bff76df09ffa038c77ad44ecced13f1125c258b2fa60fd229002f7f1dafda15
408 -----> /Users/apple/Desktop/ml/apks/0eff4ed561b34415051fb462dc998725ddbb6abd985cbabdac2c714b9f1b0da4
409 -----> /Users/apple/Desktop/ml/apks/1a1bbe402c430afed6c97c125453091da9da26bbd5418bb36659b3614c2c898a
410 -----> /Users/apple/Desktop/ml/apks/3bdefb33f0b86a2ccf2b5a0f2c9b06718a92ca8b5ad8bf6fc57a2e47328b3b57
411 -----> /Users/apple/Desktop/ml/apks/3b849210c61e270be93fde230c8f2ba67bbdc5f6c70c77ac59369859dc81b701
412 -----> /Users/apple/Desktop/ml/apks/0fed719e9b291ba87721b1958473ca62fb164155eac8c373f257ada90ab89e10
413 -----> /Users/apple/Desktop/ml/apks/0d09485e80c6a047c39ba1dfd00a1a72f42ad8c46e66436ed3c6713ffc8f716c
414 -----> /Users/apple/Desktop/ml/apks/0e734234f73f2685d64e1134ea10437648870c21eadc0312461c39f1bcc3ea41
415 -----> /Users/apple/Desktop/ml/apks/1dc609619ec3c5a

485 -----> /Users/apple/Desktop/ml/apks/2e8f821f70be10803b053e229150fc38fbeca68b4789ca059a1197fd63ef7733
486 -----> /Users/apple/Desktop/ml/apks/2cec2b77e0d10e39956730f38a89761293aae7da4a3299d9d0d69a83f7f68ef4
487 -----> /Users/apple/Desktop/ml/apks/2bdc0e959fd300abe76a974b01535d654b08573300c1b9e4863edd1b555d19dc
488 -----> /Users/apple/Desktop/ml/apks/1a098e5cec999248f284ad035ede5c2035e65f981e3f24c2c415abc3fbfe7833
489 -----> /Users/apple/Desktop/ml/apks/1b825312a360af320c9ce3992140e84fd9fc25b68b6c8c3987cc0027020038db
490 -----> /Users/apple/Desktop/ml/apks/0ca64e19e09d8af20a7a9269aca259da3406c5fd1c3627f92fb601af2c2d7d9a
491 -----> /Users/apple/Desktop/ml/apks/01be90f0119b722d0012c269a89dd548138018695fbae64b4374f3610b0d0c99
492 -----> /Users/apple/Desktop/ml/apks/0cef323a96265638fe00c69e6fa8b353f6da05a62564a96d2c5300598a8d2733
493 -----> /Users/apple/Desktop/ml/apks/00ed8f76dc592d928b3639e44eac7615c928cdca3c85097738f608f1327741ad
494 -----> /Users/apple/Desktop/ml/apks/1eef62715053680

In [ ]:
ben_api_mat = feat_mat(b.file_list).api_matrix()
print len(ben_api_mat),len(ben_api_mat[0])

0 -----> /Users/apple/Desktop/ml/crawler/benign/com.myfitnesspal.android_v6.37.2-10948_Android-5.0.apk
1 -----> /Users/apple/Desktop/ml/crawler/benign/com.rhmsoft.fm_v2.7.7-20770706_Android-4.0.apk
2 -----> /Users/apple/Desktop/ml/crawler/benign/com.jio.join_v5.0.6.multireg-5000604_Android-4.1.apk
3 -----> /Users/apple/Desktop/ml/crawler/benign/org.freeandroidtools.root_checker_v3.1.1-2293_Android-4.0.apk
4 -----> /Users/apple/Desktop/ml/crawler/benign/com.lookout_v10.22-5091e54-10220069_Android-4.1.apk
5 -----> /Users/apple/Desktop/ml/crawler/benign/com.urbandroid.lux_v8.3-275_Android-4.0.apk
6 -----> /Users/apple/Desktop/ml/crawler/benign/com.naturalmotion.csrracing_v5.0.1-21363_Android-4.0.3.apk
7 -----> /Users/apple/Desktop/ml/crawler/benign/com.amazon.mp3_v7.8.4_307080410-307080410_Android-4.4.apk
8 -----> /Users/apple/Desktop/ml/crawler/benign/com.igg.castleclash_de_v1.3.6-1300600_Android-4.1.apk
9 -----> /Users/apple/Desktop/ml/crawler/benign/com.shazam.android_v8.5.7-180524-805

76 -----> /Users/apple/Desktop/ml/crawler/benign/com.nianticproject.ingress_v1.131.0-11310_Android-4.0.apk
error
77 -----> /Users/apple/Desktop/ml/crawler/benign/com.upwork.android_v4.19.1.1-7606_Android-4.2.apk
78 -----> /Users/apple/Desktop/ml/crawler/benign/com.ea.game.pvzfree_row_v2.2.00-99_Android-4.1.apk
79 -----> /Users/apple/Desktop/ml/crawler/benign/com.linecorp.linelite_v2.2.0-184_Android-4.1.apk
80 -----> /Users/apple/Desktop/ml/crawler/benign/com.naturalmotion.csrclassics_v3.0.1-202124_Android-4.0.apk
81 -----> /Users/apple/Desktop/ml/crawler/benign/com.adguard.android_v2.11.81-2011081_Android-4.0.3.apk
82 -----> /Users/apple/Desktop/ml/crawler/benign/com.outfit7.mytalkingtomfree_v4.7.2.91-1730_Android-4.1.apk
83 -----> /Users/apple/Desktop/ml/crawler/benign/com.nll.screenrecorder_v9.7-97_Android-5.0.apk
84 -----> /Users/apple/Desktop/ml/crawler/benign/com.keepvid.studio_v3.1.2.8-29_Android-4.0.3.apk
85 -----> /Users/apple/Desktop/ml/crawler/benign/com.jb.gokeyboardpro_v1.5

153 -----> /Users/apple/Desktop/ml/crawler/benign/com.quickoffice.android-v6.5.1.12-1503111803-Android-4.4.apk
154 -----> /Users/apple/Desktop/ml/crawler/benign/com.kabam.bigrobot_v6.1.0-845368_Android-4.0.3.apk
155 -----> /Users/apple/Desktop/ml/crawler/benign/com.vng.g6.a.zombie_v4.6.4.3-278_Android-4.1.apk
156 -----> /Users/apple/Desktop/ml/crawler/benign/com.AnatomyLearning.Anatomy3DViewer3_v2.0-215_Android-4.1.apk
157 -----> /Users/apple/Desktop/ml/crawler/benign/net.mobigame.zombietsunami_v3.8.3-84_Android-4.1.apk
158 -----> /Users/apple/Desktop/ml/crawler/benign/kingoroot.supersu_v1.2.0-10200_Android-4.0.apk
159 -----> /Users/apple/Desktop/ml/crawler/benign/com.qihoo.security.lite_v1.5.8.3031-2119_Android-4.0.3.apk
160 -----> /Users/apple/Desktop/ml/crawler/benign/com.sonyericsson.android.camera_v2.1.A.0.31-4325407_Android-8.0.apk
161 -----> /Users/apple/Desktop/ml/crawler/benign/com.nextwave.wcc2_v2.7.6-59_Android-4.1.apk
error
162 -----> /Users/apple/Desktop/ml/crawler/benign/

229 -----> /Users/apple/Desktop/ml/crawler/benign/com.rovio.gold_v2.9.27354.4757-4757_Android-4.4.apk
230 -----> /Users/apple/Desktop/ml/crawler/benign/com.toh.callrecord_v1.36.3557.173-36_Android-4.0.apk
231 -----> /Users/apple/Desktop/ml/crawler/benign/com.instagram.androidbh_v6.7.1-4436713_Android-2.3.apk
232 -----> /Users/apple/Desktop/ml/crawler/benign/com.instagram.boomerang_v1.4.7-105169577_Android-4.2.apk
233 -----> /Users/apple/Desktop/ml/crawler/benign/nz.co.codepoint.minimetro_v2.1.1-39_Android-4.1.apk
234 -----> /Users/apple/Desktop/ml/crawler/benign/com.wb.goog.mkx_v1.18.0-5132_Android-4.0.apk
235 -----> /Users/apple/Desktop/ml/crawler/benign/com.wb.goog.injustice.brawler2017_v2.3.2-7092_Android-4.4.apk
236 -----> /Users/apple/Desktop/ml/crawler/benign/com.rovio.angrybirds_v7.9.3-22679300_Android-4.1.apk
237 -----> /Users/apple/Desktop/ml/crawler/benign/com.nway.powerrangerslegacywars_v2.1.0-56_Android-4.0.apk
238 -----> /Users/apple/Desktop/ml/crawler/benign/eu.chainfire.

308 -----> /Users/apple/Desktop/ml/crawler/benign/com.qihoo.security_v4.5.7.3273-2406_Android-4.0.apk
309 -----> /Users/apple/Desktop/ml/crawler/benign/com.gameloft.android.ANMP.GloftDMHM_v5.4.1a-54120_Android-4.0.apk
310 -----> /Users/apple/Desktop/ml/crawler/benign/com.supercell.clashofclans_v10.134.18-979_Android-4.0.3.apk
311 -----> /Users/apple/Desktop/ml/crawler/benign/com.hecorat.screenrecorder.free_v5.0.3-50003_Android-5.0.apk


In [ ]:
a = feat_ex('/Users/apple/Desktop/ml/apks')
a.populate_perms()
print len(global_perms),len(global_perms[0])

In [ ]:
b = feat_ex('/Users/apple/Desktop/ml/crawler/benign/')
b.populate_perms()
print len(global_perms),len(global_perms[0])

In [ ]:
global_perms = list(set(global_perms))

In [ ]:
mal_mat = feat_mat(a.file_list).perm_matrix()
print len(mal_mat),len(mal_mat[0])

In [ ]:
ben_mat = feat_mat(b.file_list).perm_matrix()
print len(ben_mat),len(ben_mat[0])

In [ ]:
apk_matrix = np.array(mal_mat[:-5] + ben_mat[:-4])
print apk_matrix.shape

In [ ]:
np.save('/Users/apple/Desktop/ml/crawler/apk_matrix.npy',apk_matrix)

In [ ]:
def extractor(apk_dir_path):
    apk_folder = os.listdir(apk_dir_path)
    print ( apk_dir_path)
    for ix in range(len(apk_folder)):
        apk_file_path = os.path.join(apk_dir_path,apk_folder[ix])
        app = apk.APK(apk_file_path)
        print feat_intents(app)

In [ ]:
extractor('/Users/apple/Desktop/ml/apks/')

In [ ]:
from sklearn.externals import joblib
feature_path = '/Users/apple/Desktop/ml'
feature_path = os.path.join(feature_path, 'features.pkl')
feature_vector = joblib.load(feature_path)

In [ ]:
import pandas as pd
train = pd.read_csv('/Users/apple/Downloads/mobile_malware_detection-master/program/csv/final_training_set.csv')
train.head(2)

In [3]:
apk_matrix = np.load('./crawler/apk_matrix.npy')
apk_matrix.shape

(1000, 1576)

In [4]:
label_matrix = np.zeros(apk_matrix.shape[0],dtype=int)
label_matrix[500:]=1
print label_matrix.shape,label_matrix[:20],label_matrix[500:520]

(1000,) [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(apk_matrix, label_matrix, test_size=0.2)

print(X_train.shape,y_train[:20])
print(y_train.shape)

((800, 1576), array([1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0]))
(800,)


/Users/apple/Desktop/ml/mlenv/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
model = Sequential()
model.add(Dense(500, input_dim=1576, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(250, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,validation_data=(X_test,y_test),
          epochs=5,batch_size=128, shuffle='batch', verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 500)               788500    
_________________________________________________________________
dropout_13 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 250)               125250    
_________________________________________________________________
dropout_14 (Dropout)         (None, 250)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 125)               31375     
_________________________________________________________________
dropout_15 (Dropout)         (None, 125)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 126       
Total para